## Overview

This notebook is intended to:
- Demonstrate how to access time series data for meters and SCADAs using Awesense's Energy Data Model (EDM) SQL API.

Please refer to the [main_concepts.ipynb](../2_main_concepts/main_concepts.ipynb) notebook for a high-level introduction to and simpler examples of the core views and functions available in Awesense's Energy Data Model (EDM).

---

## Set up

In [293]:
import getpass
import plotly.express as px
import plotly.io as pio
import time
import urllib.parse
import pandas as pd

# Allow charts to persist between notebook sessions.
pio.renderers.default='notebook'

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [294]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address: ········

EDM login information
Username: ········
Password: ········
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


---

## Time Series
Various temporary views are created below to optimize query performance within the notebook.

### Meter Time Series

*High level approach*
- Create a temporary view `grid_element_metric`, containing information on meters for grid elements and convenient access to the actual data.
- The `grid_get_downstream()` function is used to gather information of meters that are downstream of a specified grid element.
- Time series are retrieved using the `ts_data_source_select()` function.

**Downstream of a Grid** 
- Specify `grid_id` and `grid_element_id` whose downstream meter data to fetch for.

In [43]:
grid_id = input('Grid Id: ') # awefice
grid_element_id = input('Grid Element Id: ') # line_segment_57

Grid Id: North Central Zone
Grid Element Id: 12373_hvmv


In [114]:
# grid_id = 'North Central Zone'
# grid_element_id = '12373_hvmv'

In [249]:
grid_id = 'awefice'
grid_element_id = 'transformer_6'

Check when this grid was last updated.

In [186]:
%%sql

SELECT last_updated
FROM grid
WHERE grid_id = '{grid_id}';

last_updated
2021-01-01 20:00:00+00:00


In [250]:
##Finding the first branching after the first element - grid_element_id

Tree = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree = Tree.DataFrame()

AA = Tree[Tree['terminal1_cn']==Tree[Tree['grid_element_id']==grid_element_id]['terminal2_cn'].iloc[0]]
if (AA['terminal1_cn'].value_counts()>1).iloc[0]:
    print(f"First branching includes {AA}")
else:
    while ((AA['terminal1_cn'].value_counts()==1).iloc[0]):
        AA = Tree[Tree['terminal1_cn']==AA['terminal2_cn'].iloc[0]]
        if (AA['terminal1_cn'].value_counts()>1).iloc[0]:
                print(f"First branching includes {AA}")
                break
                


First branching includes      grid_element_id terminal1_cn terminal2_cn
36  line_segment_136      con_241      con_350
58   line_segment_89      con_241      con_242


In [78]:
# %%sql 
# select * from grid_get_downstream('{grid_id}', '{grid_element_id}')
# limit 10;

In [327]:
grid_id = 'awefice'
grid_element_id = 'transformer_2'

#Retrieving elements, connections and phases from all elements downstream
Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree2 = Tree2.DataFrame()

#Merging elements that come one following the other
BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

#Tracing elements where the phases change from ABC to non-ABC
bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
bif = bif.reset_index()

In [330]:
#Finding number of meters and the sum of their consumption in each cluster beneath ABC elements

grid_id = 'awefice'
grid_element_id = 'transformer_2'

#Retrieving elements, connections and phases from all elements downstream
Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree2 = Tree2.DataFrame()

#Merging elements that come one following the other
BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

#Tracing elements where the phases change from ABC to non-ABC
bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
bif['Meters_in_cluster'] = 0
bif['Total_consumption'] = 0
bif = bif.reset_index()

#Calculating consumption and munber of meters in each cluster
for ii in range(0,len(bif['grid_element_id_lower'])):
#    print(bif['grid_element_id_lower'].iloc[ii])
    grid_element_id = bif['grid_element_id_lower'].iloc[ii]
    
    EE = %sql CREATE OR REPLACE TEMPORARY VIEW grid_element_metric AS SELECT grid_id, grid_element_id, phases, type, provider, direction, friendly_id, metric_key AS metric, valid, timestamp, value \
    FROM grid_element_data_source geds \
    JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key \
        ON true \
    LEFT JOIN ts_data_source_select(grid_element_data_source_id, metric_key) AS ts \
        ON true;

    FF = %sql CREATE OR REPLACE TEMPORARY VIEW meter_data_source2 AS \
        SELECT meter.grid_id,\
                meter.grid_element_id,\
                geds.grid_element_data_source_id,\
                geds.friendly_id,\
                geds.phases,\
                geds.provider,\
                metric_key as metric,\
                lower(geds.valid) as start_time,\
                upper(geds.valid) as end_time\
        FROM grid_get_downstream('{grid_id}', '{grid_element_id}') AS meter\
        LEFT JOIN grid_element_data_source geds\
            ON meter.grid_element_id = geds.grid_element_id\
            AND meter.grid_id = geds.grid_id\
            AND geds.type = 'CONSUMER'\
        JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key\
            ON true\
        WHERE meter.type = 'Meter';

    GG = %sql CREATE OR REPLACE TEMPORARY VIEW meter_consumption2 AS\
    SELECT meter.grid_id,\
            meter.grid_element_id,\
            meter.friendly_id,\
            timestamp,\
            value AS kWh,\
            meter.phases\
    FROM meter_data_source2 meter\
    LEFT JOIN grid_element_metric gem\
        ON gem.grid_id = meter.grid_id\
        AND gem.grid_element_id = meter.grid_element_id\
    WHERE gem.metric = 'kWh'\
       AND gem.type = 'CONSUMER';

    II = %sql CREATE OR REPLACE TEMPORARY VIEW ts_stats2 AS\
        SELECT SUM(kWh) AS kWh, MIN(timestamp) AS start_timerange, MAX(timestamp) AS end_timerange\
        FROM meter_consumption2;

    JJ = %sql select COUNT(DISTINCT grid_element_id) FROM meter_data_source2;
    KK = %sql SELECT SUM(kWh) AS kWh FROM meter_consumption2;
    
    bif.loc[ii,'Meters_in_cluster'] = JJ[0][0]
    bif.loc[ii,'Total_consumption'] = KK[0][0]

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_24179/3654283460.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_24179/3654283460.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_24179/3654283460.py:36: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '235206.54450411018' has dtype incompatibl

In [331]:
bif

,index,grid_element_id_upper,terminal1_cn_upper,terminal2_cn_upper,phases_upper,grid_element_id_lower,terminal1_cn_lower,terminal2_cn_lower,phases_lower,Meters_in_cluster,Total_consumption
0,1,line_segment_1,con_365,con_60,ABC,switchable_element_25,con_60,con_62,C,3,235206.544504
1,6,line_segment_12,con_344,con_346,ABC,switchable_element_15,con_346,con_32,A,3,263749.990827
2,9,line_segment_14,con_26,con_29,ABC,switchable_element_30,con_29,con_84,B,3,287896.044378
